In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
from sklearn import linear_model

sns.set(rc={"figure.figsize": (13, 10)})

import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data = pd.read_csv("./data/train.csv").set_index("id")
valid_data = pd.read_csv("./data/test.csv").set_index("id")
datasets = [train_data, valid_data]
train_data.shape, valid_data.shape

In [ ]:
train_data.head()

In [ ]:
TARGET = "failure"
SEED = 0

In [ ]:
# load csv and set id column as index

# lable encode categorical columns
cat_cols = ['product_code', 'attribute_0', 'attribute_1']

for data in datasets:
    for col in cat_cols:
        data[col] = data[col].astype('category').cat.codes

train_data.head()

In [ ]:
print("NAs in adild data: ", valid_data.isna().sum().sum(), "\n")
train_data.isna().sum()

In [ ]:
for data in datasets:
    for col in data.filter(regex="measurement_").columns.tolist():
        data[col] = data.groupby(["product_code", "attribute_0", "attribute_1"])[col].transform(lambda x: x.fillna(x.median()))
        
    data['loading'] = data.groupby("product_code")["loading"].transform(lambda x: x.fillna(x.median()))

print(f"Train NAs: {train_data.isna().sum().sum()}\nTest NAs: {valid_data.isna().sum().sum()}")

In [ ]:
train_copy = train_data.copy(deep=True)
valid_copy = valid_data.copy(deep=True)
train_copy["origin"] = "train"
valid_copy["origin"] = "valid"
valid_copy[TARGET] = -1

train_val_copy = pd.concat([train_copy, valid_copy], axis=0)

del train_copy, valid_copy
gc.collect()